In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import utils as utils

In [2]:
train_path = 'train.csv'
val_path = 'val.csv'

train_data = pd.read_csv(train_path)
train_data['label'] = (train_data['label'] == 'self.SuicideWatch').astype(int)

val_data=pd.read_csv(val_path)
val_data['label'] = (val_data['label'] == 'self.SuicideWatch').astype(int)
train_data.describe()

,label
count,45706.000000
mean,0.187459
std,0.390284
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [3]:
def load_glove_vectors(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

# Path to the GloVe file
glove_path = 'glove.6B.100d.txt'  # Adjust this path to where you've saved your GloVe data
embeddings_index = load_glove_vectors(glove_path)

In [4]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])

#saving the tokenizer
with open('LSTMtokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Convert text to sequences and pad them
max_length = max([len(s.split()) for s in train_data['text']])
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
val_sequences = tokenizer.texts_to_sequences(val_data['text'])

X_data = pad_sequences(train_sequences, maxlen=max_length)
X_val = pad_sequences(val_sequences, maxlen=max_length)

y_val = val_data['label']
#split to train set and test set
X_train, X_test, y_train, y_test = train_test_split(X_data, train_data['label'],test_size=0.2, random_state=42)

In [5]:
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size

# Create an embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
def build_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False),
        LSTM(200, activation='tanh'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
results = {'precision': [], 'recall': [], 'f1': [], 'accuracy': []}

#loop through each percentage
for fraction in [0.999]:
    model = build_model()  # Recreate the model for each iteration
    partial_X_train, _, partial_y_train, _ = train_test_split(X_train, y_train,train_size=fraction)
    model.fit(partial_X_train, partial_y_train, epochs=8, batch_size=80, verbose=1,validation_data=(X_val, y_val))

    # Predict on X_test
    y_pred_prob = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred_prob, axis=1)

Epoch 1/8
457/457 [==============================] - 154s 330ms/step - loss: 0.4497 - accuracy: 0.8125 - val_loss: 0.3882 - val_accuracy: 0.8266
Epoch 2/8
457/457 [==============================] - 151s 331ms/step - loss: 0.4422 - accuracy: 0.8138 - val_loss: 0.4022 - val_accuracy: 0.8144
Epoch 3/8
457/457 [==============================] - 152s 332ms/step - loss: 0.3793 - accuracy: 0.8294 - val_loss: 0.3620 - val_accuracy: 0.8353
Epoch 4/8
457/457 [==============================] - 151s 332ms/step - loss: 0.3523 - accuracy: 0.8417 - val_loss: 0.3451 - val_accuracy: 0.8447
Epoch 5/8
457/457 [==============================] - 151s 331ms/step - loss: 0.3371 - accuracy: 0.8471 - val_loss: 0.3396 - val_accuracy: 0.8445
Epoch 6/8
457/457 [==============================] - 151s 330ms/step - loss: 0.3278 - accuracy: 0.8521 - val_loss: 0.3281 - val_accuracy: 0.8516
Epoch 7/8
457/457 [==============================] - 151s 331ms/step - loss: 0.3193 - accuracy: 0.8548 - val_loss: 0.3240 - val_ac

In [ ]:
model.save('LSTMBinary.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
